In [3]:
# Read in audio file
import os


found_file = False
file_path = input(
    "Enter the path to an audio file. \n Please note that it must be a .wav file recorded at 8kHz:")
while(found_file == False):
    found_file = os.path.exists(file_path) and file_path[-3:] == "wav"
    if(found_file == False):
        file_path = input("That file-path was invalid, Please try again:")

In [4]:
# Push file to GCS
from google.cloud import storage

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\rockr\Desktop\Code\SRP\sample-key.json"

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    # Uploads a file to the bucket.
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )


upload_blob("srp_speech_to_text", file_path, "audio_file_for_transcription.wav")


File C:\Users\rockr\Desktop\Code\SRP\SRP Audio Files\Speech10_8k.wav uploaded to audio file.
